In [25]:
from bs4 import BeautifulSoup
import requests
from lxml import html

In [26]:
userInput='headphones noise cancellation'
url_amzn = 'https://www.amazon.in/s/ref=nb_sb_noss_2?url=search-alias%3Daps&field-keywords=' + userInput
print(url_amzn)

https://www.amazon.in/s/ref=nb_sb_noss_2?url=search-alias%3Daps&field-keywords=headphones noise cancellation


In [27]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36'}
source_code = requests.get(url_amzn, headers=headers)


In [28]:
plain_text = source_code.text
soup = BeautifulSoup(plain_text, "html.parser")
links=[]
for titles in soup.find_all('a', {'class': 'a-link-normal a-text-normal'},{'class':'a-size-medium a-color-base a-text-normal'}):
            try:
                links.append(titles)
            except AttributeError:
                continue
                
a=(links[0])
requiredLink='http://www.amazon.in'+a.get('href')
print(requiredLink)


http://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0903865M9H4LPEJ3Q8R&url=%2FEartrons-Over-Ear-Cancellation-Bluetooth-Headphones%2Fdp%2FB07WHLFL21%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dheadphones%2Bnoise%2Bcancellation%26qid%3D1601835974%26sr%3D8-1-spons%26psc%3D1&qualifier=1601835974&id=7497501057405939&widgetName=sp_atf


In [29]:
#Producing .json file associated with scraped data

import urllib.request
import urllib.parse
import urllib.error
import ssl
import json
import re

In [30]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

html = urllib.request.urlopen(requiredLink, context=ctx).read()
html= requests.get(url_amzn, headers=headers)
soup = BeautifulSoup(html.text)
html = soup.prettify('utf-8')
product_json = {}

# This block of code will help extract the <Brand> of the item

for divs in soup.findAll('div', attrs={'class': 'a-box-group'}):
    try:
        product_json['brand'] = divs['data-brand']
        break
    except:
        pass

# This block of code will help extract the <Product Title> of the item

for spans in soup.findAll('span', attrs={'id': 'productTitle'}):
    name_of_product = spans.text.strip()
    product_json['name'] = name_of_product
    break

# This block of code will help extract the <price> of the item 

for divs in soup.findAll('div'):
    try:
        price = str(divs['data-asin-price'])
        product_json['price'] = 'Rs.' + price
        break
    except:
        pass

# This block of code will help extract the <average star rating> of the product

for i_tags in soup.findAll('i',
                           attrs={'data-hook': 'average-star-rating'}):
    for spans in i_tags.findAll('span', attrs={'class': 'a-icon-alt'}):
        product_json['star-rating'] = spans.text.strip()
        break

# This block of code will help extract the <number of customer reviews> of the product

for spans in soup.findAll('span', attrs={'id': 'acrCustomerReviewText'
                          }):
    if spans.text:
        review_count = spans.text.strip()
        product_json['customer-reviews-count'] = review_count
        break

# This block of code will help extract the <details> of the product

product_json['details'] = []
for ul_tags in soup.findAll('ul',
                            attrs={'class': 'a-unordered-list a-vertical a-spacing-none'
                            }):
    for li_tags in ul_tags.findAll('li'):
        for spans in li_tags.findAll('span',
                attrs={'class': 'a-list-item'}, text=True,
                recursive=False):
            product_json['details'].append(spans.text.strip())

#Extract <date of reviews>
product_json['reviewed-date'] = []
for divs in soup.findAll('span',
                           attrs={'class': 'a-size-base a-color-secondary review-date'
                           }):
    reviewed_date = divs.text.strip()
    product_json['reviewed-date'].append(reviewed_date)

# This block of code will help extract the <long reviews> of the product

product_json['long-reviews'] = []
for divs in soup.findAll('div', attrs={'data-hook': 'review-collapsed'
                         }):
    long_review = divs.text.strip()
    product_json['long-reviews'].append(long_review)

# Saving the scraped html file

with open('output_file.html', 'wb') as file:
    file.write(html)

# Saving the scraped data in json format

with open('product.json', 'w') as outfile:
    json.dump(product_json, outfile, indent=4)
print ('----------Extraction of data is complete. Check json file.----------')

----------Extraction of data is complete. Check json file.----------


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob

In [2]:
amz_reviews = pd.read_csv("amazon.csv")

In [3]:
amz_reviews.head()

,Ratings,Review_body,Date,Review_head
0,3.0 out of 5 stars,overall the quality music is awesome never had...,Reviewed in India on 14 August 2020,NOT THE BEST BLUETOOTH MODE HEADPHONE
1,2.0 out of 5 stars,"1. Anything you listen, even at 50% volume, is...",Reviewed in India on 21 August 2020,stay away!!
2,1.0 out of 5 stars,\n Noise cancellation doesn't work in this pr...,Reviewed in India on 19 August 2020,"Quality is not good , NO noise cancellation fe..."
3,5.0 out of 5 stars,"I have heard many headphones, but i choose ver...",Reviewed in India on 27 July 2020,Great and value for money product.
4,5.0 out of 5 stars,Battery Backup:\n10/10\nSound quality:\n10/10\...,Reviewed in India on 24 September 2020,Best value for money !!


In [4]:
amz_reviews.shape
(34660, 21)

amz_reviews.columns

Index(['Ratings', 'Review_body', 'Date', 'Review_head'], dtype='object')

In [24]:
#columns = amz_reviews[ 'Review_body']
#columns=list(columns)
#print(columns)

In [7]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
import string
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
import warnings
warnings.filterwarnings("ignore")
import re

In [22]:
# Sentiment Analysis without using the built-in tool

'''
1.Remove numbers, characters and white spaces from reviews
2.If review has 'not' replace 'not' and the next word together with an Antonym
3.Find the sentiScore for each word after tokenization
4.Using above find the sentiScore of the sentence

'''
class AntonymReplacer(object):
    def replace(self, word):
        ant = list()
        for syn in wn.synsets(word):
            for lemma in syn.lemmas():
                if lemma.antonyms():
                    ant.append(lemma.antonyms()[0].name())
        if len(ant) >= 1:
            return ant[0]
        else:
            return None

    def negreplace(self, string):
        i = 0
        sent = word_tokenize(string)
        len_sent = len(sent)
        words = []
        while i < len_sent:
            word = sent[i]
            if word == 'not' and i + 1 < len_sent:
                ant = self.replace(sent[i + 1])
                if ant:
                    words.append(ant)
                    i += 2
                    continue
            words.append(word)
            i += 1
        return words

lemmatizer = WordNetLemmatizer()


def penn_to_wn(tag):
    
    #Convert between the PennTreebank tags to simple Wordnet tags
    
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None


def swn_polarity(text):
    
    #Return a sentiment polarity: 0 = negative, 1 = positive
    

    sentiment = 0.0
    tokens_count = 0
    b = ""
    if "not" in text:
        x = ""
        raw_sentences = sent_tokenize(text)
        for raw_sentence in raw_sentences:
            tagged_sentence = pos_tag(word_tokenize(raw_sentence))

            for word, tag in tagged_sentence:
                wn_tag = penn_to_wn(tag)
                if wn_tag in (wn.NOUN, wn.ADJ):
                    x = x + " " + word
                elif word == 'not':
                    x = x + " " + word
                else:
                    continue
        rep = AntonymReplacer()
        a = rep.negreplace(x)
        for i in range(len(a)):
            b = b + " " + str(a[i])
    if b == "":
        b = text
    
    raw_sentences = sent_tokenize(b)
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))

        for word, tag in tagged_sentence: 
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue

            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue

            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue

            # Take the first sense, the most common
            
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())

            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
    return sentiment

score=[]
reviews_list=amz_reviews['Review_body']
for j in range(0,len(reviews_list)):
    paragraph =reviews_list[j] 

    paragraph = re.sub(r'\[[0-9]*\]', ' ', paragraph)  #removing references to a particular line denoted as for e.g[3]
    paragraph = re.sub(r'\s+', ' ', paragraph)  #removing duplicate white spaces
    paragraph

    formatted_paragraph = re.sub('[^a-zA-Z]', ' ', paragraph )  #removing all characters except a-z and A-Z
    formatted_paragraph = re.sub(r'\s+', ' ', formatted_paragraph)  #removing duplicate white spaces
    formatted_paragraph
    num = swn_polarity(paragraph)
    score.append(num)
    #print(num)

print(score)
print('-----------------')
    
'''
Clearly Positive*	"score": 0.8,
Clearly Negative*	"score": -0.6, 
Neutral	"score": 0.1, 
Mixed	"score": 0.0, 
'''
value=[]
for i in score:
    if i >= 0.5:
        value.append('pos')
    elif (i > -0.5) and (i < 0.5):
        value.append( 'neu')
    else:#score <= -0.5:
        value.append ('neg')
print(value)
 

[-1.375, -1.25, -1.75, 1.375, 3.125, 1.875, 3.75, 3.125, 1.25, 1.5, 0.0, 4.0, 2.875, 3.75, 2.0, 1.0, 1.625, 2.75, 2.5, -0.625, 3.125, 0.5, 0.5, 0.0, 1.125, 2.5, 2.375, -0.25, 0.5, 0.75, 0.125, -0.25, -0.625, 1.875, 0.75, 0.0, -0.25, 1.125]
-----------------
['neg', 'neg', 'neg', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'neu', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'neg', 'pos', 'pos', 'pos', 'neu', 'pos', 'pos', 'pos', 'neu', 'pos', 'pos', 'neu', 'neu', 'neg', 'pos', 'pos', 'neu', 'neu', 'pos']


In [9]:

# VADER sentiment analysis tool for getting Compound score.
def sentimental(sentence):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(sentence)
    score=vs['compound']
    return score


# VADER sentiment analysis tool for getting pos, neg and neu.
def sentimental_Score(sentence):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(sentence)
    score=vs['compound']
    if score >= 0.5:
        return 'pos'
    elif (score > -0.5) and (score < 0.5):
        return 'neu'
    elif score <= -0.5:
        return 'neg'


    
#reviews_list.apply(lambda x: sentimental_Score(x))
res=list(map(sentimental_Score,reviews_list))
print(res)

['pos', 'neg', 'neu', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'neu', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'neu', 'pos', 'pos', 'neu', 'neu', 'pos', 'pos', 'pos', 'pos', 'neu', 'neu', 'neu', 'pos', 'neu', 'pos', 'neu', 'neu', 'pos', 'neu']


In [23]:
pos=value.count('pos')
neg=value.count('neg')
if pos>neg:
    print('Ayy this product is for u only, pandaga chesko ;)')
else:
    print('Sry boss, this product aint for you :(')

Ayy this product is for u only, pandaga chesko ;)
